# Plan:

- ~~Извлекать статьи по ключевым словам;~~
- ~~Суммаризировать каждую статью;~~
- Суммаризация подходов из статей;
- Рерайтинг запроса;
- Статья в виде json;
- ~~Выбор подходящей LLM;~~ (https://ollama.com/library/llama3.1:8b-instruct-q6_K)
- Разработка веб-интерфейса;
- *Попробовать RAPTOR для суммаризации (https://arxiv.org/html/2401.18059v1);

Для работы с Ollama - https://github.com/ollama/ollama

Гайд по разработке ReAct Агента - https://langchain-ai.github.io/langgraph/how-tos/react-agent-from-scratch/#create-react-agent

# LangGraph ReAct agent tutorial

## Build agent

In [ ]:
from typing import Annotated, Sequence, TypedDict
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
import arxiv

class AgentState(TypedDict):
    """The state of the agent"""
    messages: Annotated[Sequence[BaseMessage], add_messages]
    

In [ ]:
from langchain_core.tools import tool
from langchain_ollama import ChatOllama

model = ChatOllama(model="llama3.1:8b-instruct-q6_K")

# Инструмент поиска на Arxiv
@tool("ArxivSearch") # name="arxiv_search", description=""
def arxiv_search_tool(query: str, max_results: int = 2):
    """Search articles on Arxiv by keywords"""
    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.Relevance
    )
    articles = []
    for result in search.results():
        articles.append({
            "title": result.title,
            "summary": result.summary,
            "url": result.entry_id
        })
    return articles

# Инструмент суммаризации текста
@tool("SummarizingTool") # name="summarize_tool", description="Summarizes text using a language model"
def summarize_tool(text: str):
    """Summarizes text using a language model"""
    prompt = f"Summarize the following article: {text}"
    summary = model(prompt)
    return summary


tools = [arxiv_search_tool, summarize_tool]
model = model.bind_tools(tools)

In [3]:
import json
from langchain_core.messages import ToolMessage, SystemMessage
from langchain_core.runnables import RunnableConfig

tools_by_name = {tool.name: tool for tool in tools}

def tool_node(state: AgentState):
    outputs = []
    for tool_call in state["messages"][-1].tool_calls:
        tool_result = tools_by_name[tool_call["name"]].invoke(tool_call["args"])
        outputs.append(
            ToolMessage(
                content=json.dumps(tool_result),
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )
    return {"messages": outputs}

def call_model(
        state: AgentState,
        config: RunnableConfig,
):
    system_prompt = SystemMessage(
        "You are a helpful AI assistant that takes a user input and summarize arxiv articles found by keywords from input. You can use tool you have for searching articles and summarizing them."
        )
    response = model.invoke([system_prompt] + state["messages"], config)
    return {"messages": response}

def should_continue(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    if not last_message.tool_calls:
        return "end"
    else:
        return "continue"
    

In [5]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "tools",
        "end": END,
    },
)

workflow.add_edge("tools", "agent")

graph = workflow.compile()


## Use agent

In [10]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, BaseMessage):
            print(f"=================== {message.__class__.__name__} ===================")
        else:
            message.pretty_print()
        print(message.content)

inputs = {"messages": [("user", "attention")]}
print_stream(graph.stream(inputs, stream_mode="values"))

=================== HumanMessage ===================
attention
=================== AIMessage ===================



C:\Users\koltu\AppData\Local\Temp\ipykernel_12644\477668848.py:16: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


=================== ToolMessage ===================
[{"title": "Attention and Self-Attention in Random Forests", "summary": "New models of random forests jointly using the attention and self-attention\nmechanisms are proposed for solving the regression problem. The models can be\nregarded as extensions of the attention-based random forest whose idea stems\nfrom applying a combination of the Nadaraya-Watson kernel regression and the\nHuber's contamination model to random forests. The self-attention aims to\ncapture dependencies of the tree predictions and to remove noise or anomalous\npredictions in the random forest. The self-attention module is trained jointly\nwith the attention module for computing weights. It is shown that the training\nprocess of attention weights is reduced to solving a single quadratic or linear\noptimization problem. Three modifications of the general approach are proposed\nand compared. A specific multi-head self-attention for the random forest is\nalso consid